In [1]:
#Imports
from tabula.io import read_pdf
import pandas as pd
import re

In [2]:
#Select and Read PDF
pdf_path = 'hospitallist.pdf'
dfs = read_pdf(pdf_path, pages='1-14')

In [3]:
#Setting up Raw Data Arrays
location_raw = []
name_raw = []

In [4]:
#Regex Expressions
postal_code_re = r'[A-Z]\d[A-Z][ ]?\d[A-Z]\d'
city_re = r'[A-Z ]{3,}'
hospital_name_re = r'[A-Za-z’\'.\s]{,60}(Hospital|Centre|Agency)'
health_authority_re = r'(\w+)( (Island |Coastal )*Health (Services |Care )*(Society|Authority))'
phone_re = r'(1-)*[(]*[0-9]{3}[)]*[ ,-]?[0-9]{3}[ ,-]?[0-9]{4}'

In [5]:
#Loading page one data

#Getting rid of carriage returns
dfs[0] = dfs[0].replace(to_replace = r'(\r)+', value = ' ', regex = True)

#Iterating through Column 1 and adding cells to location_raw
for values in dfs[0]['Hospital Location &\rAddress, Phone, Fax']:
    if (isinstance(values,str)):
        location_raw.append(values)

#Iterating through Column 2 and adding cells to name_raw
for values in dfs[0]['Hospital Name,\rHealth Authority Address']:
    if (isinstance(values,str)):
        name_raw.append(values)

In [6]:
#Converting other pages columns to string
location_str = ''
name_str = ''

#Iterating through pages 2-14
for i in range(1,14):

    #Iterating through Column 1 and adding cells to location_str
    for values in dfs[i]['Hospital Location &']:
        if isinstance(values,str):
            location_str += ' ' + values
    
    #Iterating through Column 2 and adding cells to name_str
    for values in dfs[i]['Hospital Name,']:
        if (isinstance(values,str) and values != 'Health Authority Address'):
            name_str += ' ' + values

In [7]:
#Delimiting data strings and adding to arrays
location2_raw = re.split(phone_re, location_str)
name2_raw = re.split(postal_code_re, name_str)

#Iterating through location2_raw filtering out useless information and adding rest to location_raw
for i in location2_raw:
    if ((isinstance(i,str)) and (len(i) >= 10) and ('TOLL FREE LINE' not in i)):
        location_raw.append(i)

#Iterating through name2_raw filtering out useless information and adding rest to name_raw
for i in name2_raw:
    if ((isinstance(i,str)) and (len(i) >= 5)):
        name_raw.append(i)


In [8]:
#Exception Handling
#First element of tuple includes row that needs to be changed second element includes correct row input specifically for name_raw
name_corner_cases_tuple = [['St. Joseph’s General Hospital Jane Murphy Chief Executive Officer', 
                        'St. Joseph’s General Hospital Jane Murphy Chief Executive Officer Island Health Authority']]

#Iterate through the name_raw array to find and corner cases matching
for i, name in enumerate(name_raw):
    for corner_case in name_corner_cases_tuple:
        if corner_case[0] in name:
            a,b = name.split(corner_case[0], 1)
            name_raw.remove(name)
            name_raw.insert(i, corner_case[1])
            name_raw.insert(i+1, b)

In [9]:
#Cleaning data and adding to new df
city = []
postal_code = []
hospital_name = []
hospital_authority = []
#Iterates through location_raw and adds string to city if city_re is a match, otherwise adds to postal_code if postal_code_re is a match
for values in location_raw:

    if (re.search(city_re, values)):
        city.append(re.search(city_re, values.strip()).group())
    else:
        city.append('None')

    if (re.search(postal_code_re, values)):
        postal_code.append(re.search(postal_code_re, values).group())
    else:
        postal_code.append('None')

#Iterates through name_raw and adds string to hospital_name if hospital_name_re is a match, otherwise adds to hospital_authority if health_authority_re is a match
for values in name_raw:

    if (re.search(hospital_name_re, values)):
        hospital_name.append(re.search(hospital_name_re, values).group())
    else:
        hospital_name.append('None')

    if (re.search(health_authority_re, values)):
        
        hospital_authority.append(re.search(health_authority_re, values).group())
    else:
        hospital_authority.append('None')

In [10]:
#Creating final dataframe
data = {'city':city,
        'postal_code':postal_code,
        'hospital_name':hospital_name,
        'health_authority':hospital_authority}

resultdf = pd.DataFrame(data)

In [11]:
#Converts to CSV
resultdf.to_csv('out.csv')
